# Imports

In [1]:
from pathlib import Path
import numpy as np
from applefy.detections.contrast import Contrast
import matplotlib.pyplot as plt

from s4hci.utils.data_handling import load_adi_data
from s4hci.detection_limits.applefy_wrapper import S4DataReduction
from applefy.utils import flux_ratio2mag, mag2flux_ratio

from applefy.utils.positions import center_subpixel

# Files we need

In [2]:
dataset_file = Path("/fast/mbonse/s4/30_data/HD22049_351_096_C-0679_A_.hdf5")
experiment_root_dir = Path("/fast/mbonse/s4/70_results/06_contrast_curves/02_cc_restored_models")
pre_trained_noise_model = "/fast/mbonse/s4/70_results/06_contrast_curves/01_train_each_model_once/HD22049_351_096_C-0679_A_/models/noise_model_fine_tuned.pkl"
pre_trained_normalization = "/fast/mbonse/s4/70_results/06_contrast_curves/01_train_each_model_once/HD22049_351_096_C-0679_A_/models/normalization_model.pkl"

# Load the data

In [3]:
# we need the psf template for contrast calculation
science_data, angles, raw_psf_template_data = load_adi_data(
    dataset_file,
    data_tag="object",
    psf_template_tag="psf_template",
    para_tag="header_object/PARANG")

dit_psf_template = 0.0042560
dit_science = 0.08

science_data = science_data[:, 12:-12, 12:-12]

psf_template = np.median(raw_psf_template_data, axis=0)
psf_template = psf_template - np.min(psf_template)

# Fit the FWHM

In [4]:
from astropy.modeling.functional_models import Moffat2D
from astropy.modeling import fitting

In [5]:
# Fit the data using astropy.modeling
p_init = Moffat2D(amplitude=1e4, x_0=10, y_0=10)
fit_p = fitting.LevMarLSQFitter()

y, x = np.mgrid[:psf_template.shape[0],
                :psf_template.shape[1]]
p = fit_p(p_init, x, y, psf_template)

In [6]:
fwhm = np.round(p.fwhm, 1)
fwhm

3.6

# Create fake planet experiments

In [7]:
tmp_exp_root = experiment_root_dir / dataset_file.name[:-5]
tmp_exp_root.mkdir(exist_ok=True)

In [8]:
contrast_instance = Contrast(
    science_sequence=science_data,
    psf_template=psf_template,
    parang_rad=angles,
    psf_fwhm_radius=fwhm / 2,
    dit_psf_template=dit_psf_template,
    dit_science=dit_science,
    scaling_factor=1., # A factor to account e.g. for ND filters
    checkpoint_dir=tmp_exp_root)

In [9]:
# fake planet brightness
flux_ratio_mag = 14
flux_ratio = mag2flux_ratio(flux_ratio_mag)

print("Brightness of fake planets in mag: " + str(flux_ratio_mag))
print("Planet-to-star flux ratio: " + str(flux_ratio))

Brightness of fake planets in mag: 14
Planet-to-star flux ratio: 2.5118864315095823e-06


In [10]:
center = center_subpixel(science_data[0])
separations = np.arange(0, center[0], fwhm / 2.)[2:]
separations

array([ 3.6,  5.4,  7.2,  9. , 10.8, 12.6, 14.4, 16.2, 18. , 19.8, 21.6,
       23.4, 25.2, 27. , 28.8, 30.6, 32.4, 34.2, 36. , 37.8, 39.6, 41.4,
       43.2, 45. , 46.8, 48.6, 50.4, 52.2, 54. , 55.8, 57.6])

In [11]:
num_fake_planets = 6

contrast_instance.design_fake_planet_experiments(
    flux_ratios=flux_ratio,
    num_planets=num_fake_planets,
    separations=separations,
    overwrite=True)

Overwriting existing config files.


# Create S4 model

In [12]:
s4_model = S4DataReduction(
    noise_model_file=pre_trained_noise_model,
    normalization_model_file=pre_trained_normalization,
    device = "cpu",
    work_dir = None,
    verbose = True)

In [13]:
_ = contrast_instance._run_fake_planet_experiment(
    algorithm_function=s4_model,
    exp_id="0000")

S4 model: restoring models ... [DONE]
S4 model: computing residual ... [DONE]
S4 model: computing residual ... [DONE]


# Print all setups for the config file

In [12]:
dataset_names = ["HD22049_303_199_C-0065_C_",
                 "HD22049_304_199_C-0065_C_",
                 "HD22049_310_0100_C-0656_A_",
                 "HD22049_351_096_C-0679_A_"]

In [13]:
for dataset_name in dataset_names:
    for exp_setup in list(contrast_instance.experimental_setups.keys()):
        #if exp_setup == "0000":
        #    continue
        print(dataset_name + " " + exp_setup)

HD22049_303_199_C-0065_C_ 0000
HD22049_303_199_C-0065_C_ 0001a
HD22049_303_199_C-0065_C_ 0001b
HD22049_303_199_C-0065_C_ 0001c
HD22049_303_199_C-0065_C_ 0001d
HD22049_303_199_C-0065_C_ 0001e
HD22049_303_199_C-0065_C_ 0001f
HD22049_303_199_C-0065_C_ 0002a
HD22049_303_199_C-0065_C_ 0002b
HD22049_303_199_C-0065_C_ 0002c
HD22049_303_199_C-0065_C_ 0002d
HD22049_303_199_C-0065_C_ 0002e
HD22049_303_199_C-0065_C_ 0002f
HD22049_303_199_C-0065_C_ 0003a
HD22049_303_199_C-0065_C_ 0003b
HD22049_303_199_C-0065_C_ 0003c
HD22049_303_199_C-0065_C_ 0003d
HD22049_303_199_C-0065_C_ 0003e
HD22049_303_199_C-0065_C_ 0003f
HD22049_303_199_C-0065_C_ 0004a
HD22049_303_199_C-0065_C_ 0004b
HD22049_303_199_C-0065_C_ 0004c
HD22049_303_199_C-0065_C_ 0004d
HD22049_303_199_C-0065_C_ 0004e
HD22049_303_199_C-0065_C_ 0004f
HD22049_303_199_C-0065_C_ 0005a
HD22049_303_199_C-0065_C_ 0005b
HD22049_303_199_C-0065_C_ 0005c
HD22049_303_199_C-0065_C_ 0005d
HD22049_303_199_C-0065_C_ 0005e
HD22049_303_199_C-0065_C_ 0005f
HD22049_3